<a href="https://colab.research.google.com/github/nandinisitlani2/python-forBI/blob/main/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## RFM RECENCY FREQUENCY MONETORY

In [2]:
%cd "/content/drive/MyDrive/Business Analyst course/Segmentation/RFM"

/content/drive/MyDrive/Business Analyst course/Segmentation/RFM


In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt

In [6]:
data=pd.read_csv("online_shoppers.csv",
                 encoding="ISO-8859-1").dropna()

# data prepration

In [7]:
data["sales"]=data["Quantity"]*data["UnitPrice"]

In [8]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [9]:
data["InvoiceData"]=pd.to_datetime(data["InvoiceDate"])

In [10]:
snapshot_date=data["InvoiceData"].max()+timedelta(days=1)

In [11]:
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [14]:
df=data.groupby(['CustomerID']).agg({
    'InvoiceData':lambda x:(snapshot_date-x.max()).days,
    "InvoiceNo":"count",
    "sales":"sum"
})

In [15]:
df["Monetory"]=df.sales/df.InvoiceNo

In [17]:
df=df.drop(columns=["sales"])

In [25]:
df.rename(columns={"InvoiceData":"Recency",
                   "InvoiceNo":"Frequency"},inplace=True)
df.head(1)

,Recency,Frequency,Monetory
CustomerID,,,
12346.0,326,2,0.0


In [24]:
df.head(1)

,InvoiceData,InvoiceNo,Monetory
CustomerID,,,
12346.0,326,2,0.0


## RFM PREP

In [27]:
df["F"]=pd.qcut(x=df["Frequency"],q=4,labels=range(1,5,1))
df["M"]=pd.qcut(x=df["Monetory"],q=4,labels=range(1,5,1))
df["R"]=pd.qcut(x=df["Recency"],q=4,labels=range(4,0,-1))
df.head(5)

,Recency,Frequency,Monetory,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,4,2
12349.0,19,73,24.076027,3,4,3
12350.0,310,17,19.670588,1,3,1


In [28]:
df["RFM"]=df[["R","F","M"]].sum(axis=1)

In [29]:
df.head()

,Recency,Frequency,Monetory,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12
12348.0,75,31,57.975484,2,4,2,8
12349.0,19,73,24.076027,3,4,3,10
12350.0,310,17,19.670588,1,3,1,5


In [30]:
def rfm_segment(df):
  if df["RFM"]>11:
    return "SuperStar",
  elif((df["RFM"]>=8)and ((df["RFM"]<11))):
    return "futureChampion"
  elif((df["RFM"]>=6)and ((df["RFM"]<8))):
    return "HighPotential"
  else:
    return "Low Relevance"

In [31]:
df["RFM_level"]=df.apply(rfm_segment,axis=1)
df.tail()

,Recency,Frequency,Monetory,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
18280.0,278,10,18.060000,1,3,1,5,Low Relevance
18281.0,181,7,11.545714,1,2,1,4,Low Relevance
18282.0,8,13,13.584615,1,2,4,7,HighPotential
18283.0,4,756,2.771005,4,1,4,9,futureChampion
18287.0,43,70,26.246857,3,4,3,10,futureChampion
